In [1]:
from pathlib import Path
import pandas as pd
import polars as pl
from sklearn.model_selection import KFold
import numpy as np
import warnings

# Loading and filtering data

The reviews dataset is loaded and filtered to include only reviews with score of 8 or more. The games are aggregated into baskets for each user.

In [2]:
data_path = Path('data')

data = pl.read_csv(data_path / 'bgg-26m-reviews.csv')
data = data.filter(pl.col('rating') >= 8)
data = (
    data
    .group_by(['user'])
    .agg(
        pl.col('name').alias('games')
    )
)

data = data.to_pandas()

# Methods for calculating the evaluation metrics

In [3]:
def precision_at_k(recommended_items, true_items, k):
    """ 
    Calculates precision at k for the recommended items.
    Parameters:
        recommended_items (list): List of recommended items.
        true_items (list): List of true items.
        k (int): The cutoff rank (number of recommended items to consider).
    """
    recommended_at_k = recommended_items if len(recommended_items) < k else recommended_items[:k]
    true_positives = len(set(recommended_at_k) & set(true_items))
    precision = true_positives / (len(recommended_at_k) if len(recommended_at_k) > 0 else 1)
    return precision

def recall_at_k(recommended_items, true_items, k):
    """ 
    Calculates recall at k for the recommended items.
    Parameters:
        recommended_items (list): List of recommended items.
        true_items (list): List of true items.
        k (int): The cutoff rank (number of recommended items to consider).
    """
    recommended_at_k = recommended_items if len(recommended_items) < k else recommended_items[:k]
    true_positives = len(set(recommended_at_k) & set(true_items))
    recall = true_positives / (len(true_items) if len(true_items) > 0 else 1)
    return recall

def fscore_at_k(recommended_items, true_items, k):
    """
    Calculates F1-score at k for the recommended items.
    """
    p = precision_at_k(recommended_items, true_items, k)
    r = recall_at_k(recommended_items, true_items, k)

    if p + r == 0:
        return 0.0

    return 2 * p * r / (p + r)

In [4]:
def evaluate_model(recommended_items, basket_test, k):
    """ 
    Evaluates the model's predictions using precision and recall at k.
    Parameters: 
        predictions_df (pd.DataFrame): DataFrame from get_predictions_from_rules() function.
        k (int): The cutoff rank (number of recommended items to consider).
    """
    precisions = []
    recalls = []
    fscores = []
    for basket in basket_test['games']:
        precisions.append(precision_at_k(recommended_items, basket, k=k))
        recalls.append(recall_at_k(recommended_items, basket, k=k))
        fscores.append(fscore_at_k(recommended_items, basket, k=k))

    return np.mean(precisions), np.mean(recalls), np.mean(fscores)

# Cross validation

Evaluation metrics are calculated using 5-fold CV.

In [ ]:
K=5
folds = KFold(n_splits=K, shuffle=True, random_state=42)
ks = np.linspace(1, 10, 5, dtype=int)

CV_results = {
    'ID': [],
    'k': [],
    'precision': [],
    'recall': [],
    'fscore': []
}

fold_id = 1

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    for train_index, test_index in folds.split(data):
        baskets_train = data.iloc[train_index]
        baskets_test = data.iloc[test_index]

        #Finding most frequent games in training set
        item_counts = baskets_train.explode('games')['games'].value_counts().sort_values(ascending=False)
        top_items = item_counts.index.tolist()
        
        for k in ks:
            # Recommending top-k most frequent games
            recommended_items = top_items[:k]
            
            # Calculating evaluation metrics
            precision, recall, fscore = evaluate_model(recommended_items, baskets_test, k)
            print(f"K: {k} => Precision: {precision}, Recall: {recall}, F-score: {fscore}")
            CV_results['ID'].append(fold_id)
            CV_results['k'].append(k)
            CV_results['precision'].append(precision)
            CV_results['recall'].append(recall)
            CV_results['fscore'].append(fscore)
        fold_id += 1

CV_results_df = pd.DataFrame(CV_results)
CV_results_df

K: 1 => Precision: 0.14842687629230855, Recall: 0.012258474142060832, F-score: 0.019023943132761005
K: 3 => Precision: 0.13643052735502859, Recall: 0.03119733542185261, F-score: 0.038717316480577565
K: 5 => Precision: 0.128435445112978, Recall: 0.04876960549418492, F-score: 0.05188425799511494
K: 7 => Precision: 0.12250087151825131, Recall: 0.06546642661988326, F-score: 0.06176549807597498
K: 10 => Precision: 0.11471601810629063, Recall: 0.090159280325983, F-score: 0.07124994012582375
K: 1 => Precision: 0.14816608609801984, Recall: 0.012277778403013676, F-score: 0.019145171464356705
K: 3 => Precision: 0.13420760141323446, Recall: 0.0315898846766457, F-score: 0.03867002316120648
K: 5 => Precision: 0.12723394743214803, Recall: 0.04958556408346265, F-score: 0.05198533831250667
K: 7 => Precision: 0.12170519531322764, Recall: 0.06655875656490531, F-score: 0.06196447582002509
K: 10 => Precision: 0.11397649162677198, Recall: 0.09077234806297374, F-score: 0.07123064065364004
K: 1 => Precision:

,ID,k,precision,recall,fscore
0,1,1,0.148427,0.012258,0.019024
1,1,3,0.136431,0.031197,0.038717
2,1,5,0.128435,0.048770,0.051884
3,1,7,0.122501,0.065466,0.061765
4,1,10,0.114716,0.090159,0.071250
5,2,1,0.148166,0.012278,0.019145
6,2,3,0.134208,0.031590,0.038670
7,2,5,0.127234,0.049586,0.051985
8,2,7,0.121705,0.066559,0.061964
9,2,10,0.113976,0.090772,0.071231


In [ ]:
# Save evaluation results
CV_results_df.to_csv(data_path / "baseline_CV_results.csv", index=False)